In [ ]:
import pandas as pd
# 'Requests' package allows us to gather source code from the website
import requests as req
# Regular Expression and BeautifulSoup are different methods that can get specific information from source code
# 're' package allows us to write a regular expression that can get the information with same format
import re
# 'bs' package allows us to gather information with specific labels, attributes and values
from bs4 import BeautifulSoup


# need to change because abnormally request the same URL with same user-agent will block your address
heads = heads = {"User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/94.0.4606.61 Safari/537.36"}
url = "https://fastfoodnutrition.org/fast-food-restaurants"

# open the url you enter and get HTML code 
resp = req.get(url=url,headers = heads)
html_content = resp.text
html_content

'\n<!DOCTYPE html>\n<html lang="en">\n<head>\n<link rel="preload" href="https://fonts.googleapis.com/css?family=Open+Sans:300,400,700&display=swap" as="style">\n<meta charset="UTF-8">\n<meta http-equiv="X-UA-Compatible" content="IE=edge">\n<meta name="viewport" content="width=device-width, initial-scale=1.0">\n<meta property="og:site_name" content="FastFoodNutrition.org" />\n<meta property="og:type" content="Website" />\n<meta property="og:url" content="https://fastfoodnutrition.org/fast-food-restaurants" />\n<meta property="twitter:url" content="https://fastfoodnutrition.org/fast-food-restaurants" />\n<link rel="icon" type="image/png" href="https://fastfoodnutrition.org/favicon.png" sizes="16x16">\n<meta name="google-site-verification" content="uwY4KDvAqjtaNa915c-BSMoZL72Q-vDvBDAn45cmYDA" />\n<meta property="fb:app_id" content="510431215760752" />\n<link href="https://fastfoodnutrition.org/fast-food-restaurants" rel="alternate" hreflang="en-us"><link href="https://fastfoodnutrition.or

In [ ]:
# convert the content to a more tractable format, 'html.parser' specifies that it is HTML file
soup = BeautifulSoup(html_content, 'html.parser')
# get information with label and attribute and generate a interator
elms = soup.find_all('div', {'class': 'filter_target col-6 col-sm-6 col-md-4'})
box = {}
# create a dictionary that includes the original brand name and the brand name that can be linked in URL
for i in range(len(elms)):
    binn = elms[i]
    ahref = binn.find('a').get('href')
    title = binn.find('a').get('title')
    title = title.replace(' Nutrition Facts', '')
    box[title] = ahref
print(box)

{'7-Eleven': '/7-eleven', "Applebee's": '/applebees', "Arby's": '/arbys', 'Au Bon Pain': '/au-bon-pain', "Auntie Anne's": '/auntie-annes', 'Baja Fresh': '/baja-fresh', 'Baskin-Robbins': '/baskin-robbins', 'Bibibop': '/bibibop', "BJ's": '/bjs', 'Blaze Pizza': '/blaze-pizza', 'Blimpie': '/blimpie', 'Bob Evans': '/bob-evans', 'Bojangles': '/bojangles', 'Boston Market': '/boston-market', "Bruegger's": '/brueggers', 'Buffalo Wild Wings': '/buffalo-wild-wings', 'Burger King': '/burger-king', 'BurgerFi': '/burgerfi', 'Cafe Rio': '/cafe-rio', 'California Pizza Kitchen': '/california-pizza-kitchen', "Captain D's": '/captain-ds', 'Caribou Coffee': '/caribou-coffee', "Carl's Jr": '/carls-jr', "Carrabba's": '/carrabbas', "Casey's": '/caseys', 'Cava': '/cava', 'Checkers': '/checkers', 'The Cheesecake Factory': '/cheesecake-factory', 'Chick-fil-A': '/chick-fil-a', 'Chicken Express': '/chicken-express', 'Chicken Salad Chick': '/chicken-salad-chick', "Chili's": '/chilis', 'Chipotle': '/chipotle', "Chu

In [ ]:
# get the menu of each brand
for name,suf in box.items():
    # link the brand name to the original URL to get the brand-specified website
    url = "https://fastfoodnutrition.org"
    url = url + suf
    resp = req.get(url=url,headers = heads)
    html_content = resp.text
    # write a REGULAR EXPRESSION, get the all contents conform the formula in the () and store them in an iterator 
    obj = re.compile(r'<li class="filter_target" .*? title="(?P<item>.*?)"><div class=',re.S)
    content_t = obj.finditer(html_content)
    food = []
    # store the information we get above (menu) in a list
    for i in content_t:
        food.append(i.group('item'))
    # clean the names of menu to make them can be appended to the original link
    total_content = []
    for subname in food:
        content = []
        subname = subname.lower()
        subname = subname.replace('"','')
        subname = subname.replace("'","")
        subname = subname.replace(" - "," ")
        subname = subname.replace(" – "," ")
        subname = subname.replace(" 'n"," n")
        subname = subname.replace("'n"," n")
        subname = subname.replace("'s","")
        subname = subname.replace(",","")
        subname = subname.replace("%","")
        subname = subname.replace("&reg;","")
        subname = subname.replace("&copy;","")
        subname = subname.replace("& ","")
        subname = subname.replace("/","")
        subname = subname.replace(" ","-")
        suburl = url + '/' +subname
        # open nutrition website and get detailed nutritions for each item
        resp2 = req.get(url=suburl,headers = heads)
        html_content2 = resp2.text
        obj2 = re.compile('<td title=.*?>(?P<amount>.*?)</td>',re.S)
        content_tt = obj2.finditer(html_content2)
        for c in content_tt:
            content.append(c.group('amount'))
        new_content = []
        # do not use the first item
        for each in content[1:]:
            each = each.replace("\n","")
            new_content.append(each)
        total_content.append(new_content)
    new_name_list = []
    new_data_list = []
    for n in range(len(total_content)):
        # only get the nutrition content with 11 elements (however, still have some uncleaned data)
        if len(total_content[n]) == 11:
            new_name_list.append(food[n])
            new_data_list.append(total_content[n])
        else:
            continue
    columnslist = ['calories','calfromfat','fat','satfat','transfat','cho','sodium','charbo','dietfac','sugar','protein']
    mcdata = pd.DataFrame(new_data_list,columns=columnslist,index=new_name_list)
    mcdata['brand'] = name
    filename = str(name)+'.csv'
    mcdata.to_csv(filename)
    print('OK:',name)

In [ ]:
mcdata.head()

,calories,calfromfat,fat,satfat,transfat,cho,sodium,charbo,dietfac,sugar,protein,brand
5 React Fruit Slurpee,66,?,?,?,?,0mg,6mg,18g,0g,18g,0g,7-Eleven
Black Iced Coffee,5,?,?,?,?,0mg,25mg,1g,0g,?,0g,7-Eleven
Blue Lightning Blast Slurpee,70,?,?,?,?,0mg,20mg,18g,0g,18g,0g,7-Eleven
Cherry Orange Blitz Slurpee,110,?,?,?,?,0mg,25mg,28g,0g,27g,0g,7-Eleven
Coca Cola Classic Slurpee,179,?,?,?,?,0mg,16mg,49g,0g,49g,0g,7-Eleven
